In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os
from sklearn import metrics
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from scipy.spatial.distance import cdist
import plotly.express as px
from plotly.subplots import make_subplots
drive.mount('/content/gdrive/') 

Mounted at /content/gdrive/


In [2]:
!gdown --id 1-8Nze1uyLwTuALzlf3HuCctnprwjhPgq

Downloading...
From: https://drive.google.com/uc?id=1-8Nze1uyLwTuALzlf3HuCctnprwjhPgq
To: /content/endomondo_preprocessed.csv
3.36GB [00:29, 115MB/s]


In [ ]:
%%time
# https://drive.google.com/file/d/1-8Nze1uyLwTuALzlf3HuCctnprwjhPgq/view?usp=sharing
df = pd.read_csv('endomondo_preprocessed.csv')
print(len(df))

65500
CPU times: user 29.5 s, sys: 6.78 s, total: 36.3 s
Wall time: 1min 15s


In [ ]:
df.head(3)

,id,userId,gender,sport,duration,calories,distance,avg_heart_rate,longitude,latitude,altitude,timestamp,heart_rate,speed,url,derived_distance,derived_speed,time_elapsed,validate,avg_alti,change_alti,max_alti,min_alti,diff_alti,avg_speed
0,396826535,10921915,male,bike,7589,1562.0,53.88,152.65,"[24.64977040886879, 24.65014273300767, 24.6509...","[60.173348765820265, 60.173239801079035, 60.17...","[41.6, 40.6, 40.6, 38.4, 37.0, 34.0, 34.0, 34....","[1408898746, 1408898754, 1408898765, 140889877...","[100, 111, 120, 119, 120, 116, 125, 128, 131, ...","[6.8652, 16.4736, 19.1988, 20.4804, 31.3956, 3...",https://www.endomondo.com/users/10921915/worko...,"[0.023920359645758085, 0.0513686868631109, 0.0...","[10.764161840591138, 16.81157024610902, 15.919...","[8, 11, 13, 16, 6, 23, 16, 23, 29, 23, 24, 24,...",True,43.0712,820.6,74.2,14.6,59.6,25.568980
1,392337038,10921915,male,bike,4440,908.0,32.61,147.71,"[24.649855233728886, 24.65015547350049, 24.650...","[60.173247596248984, 60.17320962622762, 60.172...","[38.4, 39.0, 39.0, 38.2, 36.8, 36.8, 36.8, 35....","[1408221682, 1408221687, 1408221699, 140822170...","[100, 105, 111, 110, 108, 115, 126, 130, 132, ...","[9.0792, 13.284, 15.9336, 10.9476, 16.1676, 30...",https://www.endomondo.com/users/10921915/worko...,"[0.01714931222251621, 0.047614484656155226, 0....","[12.34750480021167, 14.284345396846566, 17.330...","[5, 12, 8, 4, 5, 6, 4, 4, 5, 13, 7, 17, 4, 10,...",True,35.7248,454.2,67.8,17.2,50.6,27.129599
2,386729739,10921915,male,bike,4519,921.0,32.70,147.02,"[24.649874344468117, 24.650242729112506, 24.65...","[60.173286236822605, 60.17323946580291, 60.173...","[28.6, 29.4, 29.4, 29.4, 28.0, 27.0, 25.2, 23....","[1407432042, 1407432048, 1407432056, 140743206...","[99, 105, 113, 109, 112, 116, 116, 114, 114, 1...","[7.4988, 15.4008, 17.8668, 21.5712, 24.1416, 3...",https://www.endomondo.com/users/10921915/worko...,"[0.02104899112126496, 0.03733382239564792, 0.0...","[12.629394672758975, 16.800220078041566, 20.17...","[6, 8, 9, 4, 13, 4, 3, 4, 13, 10, 13, 13, 7, 1...",True,32.6104,481.4,64.0,12.4,51.6,26.651989


# Cluster for Running

In [ ]:
df_sub = df[df['sport']=='run'].copy()
df_sub.reset_index(drop=1,inplace=True)
df_sub

,id,userId,gender,sport,duration,calories,distance,avg_heart_rate,longitude,latitude,altitude,timestamp,heart_rate,speed,url,derived_distance,derived_speed,time_elapsed,validate,avg_alti,change_alti,max_alti,min_alti,diff_alti,avg_speed
0,321063199,4969375,male,run,3888,436.0,8.650000,122.104,"[6.8854929, 6.8853678, 6.8851621, 6.8848205, 6...","[52.2226809, 52.222727, 52.2228258, 52.2228606...","[-173.8, -151.2, -161.6, -165.4, -168.6, -172....","[1397079203, 1397079210, 1397079218, 139707922...","[80, 81, 94, 100, 102, 112, 108, 114, 110, 109...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.024692396657788666, 0.02062475820692565, 0....","[12.698946852577029, 10.911666227748753, 10.05...","[7, 8, 8, 8, 7, 8, 8, 8, 8, 7, 8, 8, 8, 8, 7, ...",True,-31.706000,762.000000,41.2000,-360.80000,402.00000,8.255823
1,303565793,4969375,male,run,4204,577.0,14.370000,146.994,"[6.9144073, 6.9142929, 6.9141539, 6.9140268, 6...","[52.2111711, 52.2112631, 52.2114064, 52.211608...","[57.8, 57.6, 57.0, 56.4, 55.8, 55.2, 54.4, 53....","[1393908533, 1393908541, 1393908549, 139390855...","[60, 62, 92, 92, 132, 150, 150, 159, 159, 161,...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.012866693107454877, 0.018551828170520174, 0...","[5.790011898354694, 7.923126352508781, 8.85172...","[8, 8, 9, 8, 9, 8, 9, 8, 8, 9, 8, 9, 8, 9, 8, ...",True,41.671200,205.800000,57.8000,23.20000,34.60000,12.202044
2,296982347,4969375,male,run,6503,1025.0,22.220000,134.918,"[6.8678543, 6.8678634, 6.8675429, 6.8672183, 6...","[52.1936673, 52.1934354, 52.1931993, 52.192873...","[35.4, 35.2, 34.6, 34.2, 35.0, 35.2, 34.8, 34....","[1392480163, 1392480176, 1392480189, 139248020...","[75, 101, 116, 120, 124, 126, 127, 129, 126, 1...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.0258024349522409, 0.034169705873918786, 0.0...","[7.1452896790820954, 9.460093775795364, 10.251...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...",True,34.125600,248.000000,44.8000,24.60000,20.20000,12.104143
3,295890426,4969375,male,run,5118,775.0,16.980000,132.414,"[6.9143328, 6.9146396, 6.9148949, 6.9151568, 6...","[52.2112195, 52.2110264, 52.2108135, 52.210601...","[63.0, 65.2, 66.0, 66.2, 65.8, 65.8, 67.0, 67....","[1392180426, 1392180436, 1392180446, 139218045...","[58, 83, 112, 115, 117, 116, 141, 121, 120, 11...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.03005678592559137, 0.029397223648873875, 0....","[10.820442933212894, 10.685701145251175, 10.36...","[10, 10, 10, 11, 10, 10, 10, 11, 10, 10, 10, 1...",True,45.097200,191.800000,67.0000,27.00000,40.00000,11.824599
4,294163731,4969375,male,run,3035,445.0,9.670000,125.346,"[6.9133737, 6.9132722, 6.913217, 6.9131066, 6....","[52.2111481, 52.2111209, 52.21119, 52.2112981,...","[61.4, 56.6, 53.2, 49.6, 46.0, 42.4, 38.8, 35....","[1391663545, 1391663551, 1391663557, 139166356...","[56, 62, 75, 80, 80, 96, 107, 108, 113, 117, 1...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.008947091230656174, 0.009208079362029106, 0...","[5.368254738393704, 6.557825426283524, 7.28499...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6, 6, 6, ...",True,48.845600,252.000000,71.2000,0.40000,70.80000,11.395391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36896,298971490,69,male,run,2047,623.0,6.013510,155.294,"[11.76463108509779, 11.76454659551382, 11.7644...","[55.24085112847388, 55.24080477654934, 55.2407...","[22.4, 22.4, 21.6, 21.8, 21.6, 21.2, 21.4, 21....","[1392921688, 1392921692, 1392921696, 139292170...","[99, 100, 104, 111, 119, 122, 128, 128, 128, 1...",NaN,https://www.endomondo.com/users/69/workouts/29...,"[0.0074356460011203145, 0.01056021379557203, 0...","[6.692081401008283, 9.190715975634406, 9.97964...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, ...",True,21.016800,213.800000,38.6000,13.40000,25.20000,10.481452
36897,235763396,69,male,run,7387,1374.0,21.310540,134.108,"[11.757076978683472, 11.757056526839733, 11.75...","[55.24856

In [ ]:
# select faetures
feature = ['id','distance','change_alti','diff_alti','avg_alti','min_alti','max_alti']
df_cluster = df_sub[feature]

train = df_cluster.drop('id',axis=1)
t= preprocessing.StandardScaler().fit(train)
train=t.transform(train)
train

array([[-0.53128381,  1.55083853,  5.31533641, -0.73711031, -1.88479699,
        -0.55548346],
       [ 0.35110375, -0.39426458, -0.28949619, -0.47056389, -0.45780692,
        -0.49740613],
       [ 1.5620727 , -0.24668574, -0.5091739 , -0.49797367, -0.45260436,
        -0.54288837],
       ...,
       [ 4.83868316,  0.47512167, -0.51527606, -0.48659797, -0.43402376,
        -0.52679465],
       [ 0.6132692 , -0.18093972,  0.18342109, -0.4926745 , -0.51949452,
        -0.44702579],
       [-1.02241507,  0.09461253, -0.34804351, -0.56636838, -0.54886929,
        -0.59656617]])

In [ ]:
%%time
ch_score = []
ss_score = []
meandistortions = []
# try k from 2 to 15
for k in range(2,15):
    clf = KMeans(n_clusters=k)
    pred = clf.fit_predict(train)
    ch = metrics.calinski_harabasz_score(train,pred)
    ss = metrics.silhouette_score(train,pred)
    ch_score.append(ch)
    ss_score.append(ss)
    meandistortions.append(sum(np.min(cdist(train, clf.cluster_centers_, 'euclidean'), axis=1)) / train.shape[0])

CPU times: user 5min 14s, sys: 1min 36s, total: 6min 51s
Wall time: 4min 2s


In [ ]:
fig = make_subplots(rows=1, cols=3,subplot_titles=("Calinski Harabasz Score", "Elbow Score",'Silhouette Score'))

fig.add_trace(go.Scatter(x=list(range(2,15)), y=ch_score,
                    mode='lines+markers',),row=1,col=1)

fig.add_trace(go.Scatter(x=list(range(2,15)), y=ss_score,
                    mode='lines+markers',
                    name='lines'),row=1,col=3,)

fig.add_trace(go.Scatter(x=list(range(2,15)), y=meandistortions,
                    mode='lines+markers',
                    name='lines'),row=1,col=2,)

fig.update_layout(height=600, width=1200, title_text="Running: Number of K",showlegend=False)
fig.show()

In [ ]:
# choose the best number of k
clf = KMeans(n_clusters=6)
pred = clf.fit_predict(train)
pred = [str(i) for i in pred]

In [ ]:
# use tsne to plot the clusters
tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(train)
projections_df = pd.DataFrame(projections)
projections_df['Cluster']=pred

In [ ]:
fig = px.scatter(
    projections_df, x=0, y=1,
    color='Cluster',
    labels={'color': 'Cluster'},
    category_orders={"Cluster": ["0", "1", "2", "3","4",'5']}
)

fig.update_layout(height=600, width=1200,title='TSNE: Running Route Clustering',font_size=22)
fig.show()

In [ ]:
t = df_cluster.copy()
t['K_means'] = pred
t = t.groupby('K_means').mean()

In [ ]:
x_axis = ['Cluster '+str(i) for i in range(len(t))]
fig = go.Figure(data=[
    go.Bar(name='Distance', x=x_axis, y=t['distance']*100),
    go.Bar(name='Change of Altitude', x=x_axis, y=t['change_alti']),
    go.Bar(name='Diff_Altitude', x=x_axis, y=t['diff_alti']),
    go.Bar(name='Avg_Altitude', x=x_axis, y=t['avg_alti'])
])
# Change the bar mode
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))
fig.update_layout(barmode='group',font_size=22,height=600,width=1200,title = 'Clusters of Running')
fig.show()

In [ ]:
df_sub['Cluster']=pred
df_run = df_sub.copy()

# Cluster for Biking

In [ ]:
df_sub = df[df['sport']=='bike'].copy()
df_sub.reset_index(drop=1,inplace=True)
df_sub

,id,userId,gender,sport,duration,calories,distance,avg_heart_rate,longitude,latitude,altitude,timestamp,heart_rate,speed,url,derived_distance,derived_speed,time_elapsed,validate,avg_alti,change_alti,max_alti,min_alti,diff_alti,avg_speed
0,396826535,10921915,male,bike,7589,1562.000,53.880000,152.650,"[24.64977040886879, 24.65014273300767, 24.6509...","[60.173348765820265, 60.173239801079035, 60.17...","[41.6, 40.6, 40.6, 38.4, 37.0, 34.0, 34.0, 34....","[1408898746, 1408898754, 1408898765, 140889877...","[100, 111, 120, 119, 120, 116, 125, 128, 131, ...","[6.8652, 16.4736, 19.1988, 20.4804, 31.3956, 3...",https://www.endomondo.com/users/10921915/worko...,"[0.023920359645758085, 0.0513686868631109, 0.0...","[10.764161840591138, 16.81157024610902, 15.919...","[8, 11, 13, 16, 6, 23, 16, 23, 29, 23, 24, 24,...",True,43.0712,820.6,74.2,14.6,59.6,25.568980
1,392337038,10921915,male,bike,4440,908.000,32.610000,147.710,"[24.649855233728886, 24.65015547350049, 24.650...","[60.173247596248984, 60.17320962622762, 60.172...","[38.4, 39.0, 39.0, 38.2, 36.8, 36.8, 36.8, 35....","[1408221682, 1408221687, 1408221699, 140822170...","[100, 105, 111, 110, 108, 115, 126, 130, 132, ...","[9.0792, 13.284, 15.9336, 10.9476, 16.1676, 30...",https://www.endomondo.com/users/10921915/worko...,"[0.01714931222251621, 0.047614484656155226, 0....","[12.34750480021167, 14.284345396846566, 17.330...","[5, 12, 8, 4, 5, 6, 4, 4, 5, 13, 7, 17, 4, 10,...",True,35.7248,454.2,67.8,17.2,50.6,27.129599
2,386729739,10921915,male,bike,4519,921.000,32.700000,147.020,"[24.649874344468117, 24.650242729112506, 24.65...","[60.173286236822605, 60.17323946580291, 60.173...","[28.6, 29.4, 29.4, 29.4, 28.0, 27.0, 25.2, 23....","[1407432042, 1407432048, 1407432056, 140743206...","[99, 105, 113, 109, 112, 116, 116, 114, 114, 1...","[7.4988, 15.4008, 17.8668, 21.5712, 24.1416, 3...",https://www.endomondo.com/users/10921915/worko...,"[0.02104899112126496, 0.03733382239564792, 0.0...","[12.629394672758975, 16.800220078041566, 20.17...","[6, 8, 9, 4, 13, 4, 3, 4, 13, 10, 13, 13, 7, 1...",True,32.6104,481.4,64.0,12.4,51.6,26.651989
3,371302193,10921915,male,bike,4422,889.000,32.650000,145.914,"[24.65018338523805, 24.650422520935535, 24.650...","[60.17324591986835, 60.17319998703897, 60.1730...","[21.2, 22.0, 22.0, 20.6, 19.8, 19.4, 17.6, 15....","[1405100572, 1405100576, 1405100582, 140510059...","[100, 98, 104, 104, 107, 109, 108, 105, 105, 1...","[16.4124, 11.682, 18.162, 21.7728, 24.408, 26....",https://www.endomondo.com/users/10921915/worko...,"[0.014204602439679812, 0.028542833923925526, 0...","[12.78414219571183, 17.125700354355313, 20.887...","[4, 6, 9, 5, 10, 4, 4, 6, 10, 10, 5, 10, 11, 4...",True,22.1912,522.4,56.4,4.0,52.4,27.306088
4,366977747,10921915,male,bike,6770,1222.000,46.370000,143.224,"[24.64970293454826, 24.65009386651218, 24.6508...","[60.173228066414595, 60.173104433342814, 60.17...","[33.6, 32.4, 32.4, 32.0, 29.2, 27.8, 27.8, 27....","[1404415397, 1404415406, 1404415421, 140441544...","[91, 100, 107, 105, 104, 108, 120, 114, 113, 1...","[3.96, 13.3524, 19.5012, 25.3764, 32.6124, 26....",https://www.endomondo.com/users/10921915/worko...,"[0.02565751019948074, 0.055649820287697715, 0....","[10.263004079792296, 13.355956869047452, 19.92...","[9, 15, 23, 5, 10, 18, 12, 17, 7, 12, 19, 17, ...",True,48.5860,701.6,79.4,21.2,58.2,24.824423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25151,571917798,69,male,bike,1190,336.478,9.709200,151.938,"[166.948185, 166.94821, 166.948254, 166.948309...","[-11.639523, -11.639488, -11.639445, -11.63939...","[6.4, 6.2, 6.0, 5.8, 5.4, 5.2, 5.0, 4.8, 4.4, ...","[1438367060, 1438367062, 1438367064, 143836706...","[105, 105, 106, 107, 108, 109, 109, 111, 112, ...",NaN,https://www.endomondo.com/users/69/workouts/57...,"[0.004755375022161955, 0.00677446292991385, 0....","[8.559675039891518, 12.19403327384493, 14.1870...","[2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, ...",True,21.2

In [ ]:
feature = ['id','distance','change_alti','diff_alti','avg_alti','min_alti','max_alti']
df_cluster = df_sub[feature]

train = df_cluster.drop('id',axis=1)
t= preprocessing.StandardScaler().fit(train)
train=t.transform(train)
train

array([[ 0.95804389,  0.87212916, -0.19680713, -0.40553436, -0.418526  ,
        -0.41297738],
       [-0.07329   , -0.07990473, -0.30097974, -0.44338141, -0.40400579,
        -0.4417235 ],
       [-0.06892611, -0.00922972, -0.289405  , -0.45942612, -0.43081233,
        -0.45879151],
       ...,
       [ 0.45631307,  0.40858427, -0.03939076, -0.44006984, -0.47213909,
        -0.39501105],
       [ 0.46305233,  0.33842894, -0.06948507, -0.37105039, -0.37161455,
        -0.32584069],
       [ 0.44620619,  0.35297967, -0.06948507, -0.38765562, -0.40400579,
        -0.35189186]])

In [ ]:
%%time
ch_score = []
ss_score = []
meandistortions = []
for k in range(2,15):
    clf = KMeans(n_clusters=k)
    pred = clf.fit_predict(train)
    ch = metrics.calinski_harabasz_score(train,pred)
    ss = metrics.silhouette_score(train,pred)
    ch_score.append(ch)
    ss_score.append(ss)
    meandistortions.append(sum(np.min(cdist(train, clf.cluster_centers_, 'euclidean'), axis=1)) / train.shape[0])

CPU times: user 2min 28s, sys: 55.6 s, total: 3min 24s
Wall time: 1min 50s


In [ ]:
fig = make_subplots(rows=1, cols=3,subplot_titles=("Calinski Harabasz Score", "Elbow Score",'Silhouette Score'))

fig.add_trace(go.Scatter(x=list(range(2,15)), y=ch_score,
                    mode='lines+markers',),row=1,col=1)

fig.add_trace(go.Scatter(x=list(range(2,15)), y=ss_score,
                    mode='lines+markers',
                    name='lines'),row=1,col=3,)

fig.add_trace(go.Scatter(x=list(range(2,15)), y=meandistortions,
                    mode='lines+markers',
                    name='lines'),row=1,col=2,)

fig.update_layout(height=600, width=1200, title_text="Biking: Number of K",showlegend=False)
fig.show()

In [ ]:
clf = KMeans(n_clusters=5)
pred = clf.fit_predict(train)
pred = [str(i) for i in pred]

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(train)

In [ ]:
projections_df = pd.DataFrame(projections)
projections_df['Cluster']=pred

In [ ]:
fig = px.scatter(
    projections_df, x=0, y=1,
    color='Cluster',
    labels={'color': 'Cluster'},
    category_orders={"Cluster": ["0", "1", "2", "3","4"]}
)

fig.update_layout(height=600, width=1200,title='TSNE: Biking Route Clustering',font_size=22)
fig.show()

In [ ]:
t = df_cluster.copy()
t['K_means'] = pred
t = t.groupby('K_means').mean()

In [ ]:
x_axis = ['Cluster '+str(i) for i in range(len(t))]
fig = go.Figure(data=[
    go.Bar(name='Distance', x=x_axis, y=t['distance']*100),
    go.Bar(name='Change of Altitude', x=x_axis, y=t['change_alti']),
    go.Bar(name='Diff_Altitude', x=x_axis, y=t['diff_alti']),
    go.Bar(name='Avg_Altitude', x=x_axis, y=t['avg_alti'])
])
# Change the bar mode
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))
fig.update_layout(barmode='group',font_size=22,height=600,width=1200,title = 'Cluster of Biking')
fig.show()

In [ ]:
df_sub['Cluster']=pred
df_bike = df_sub.copy()

# Cluster for mountain bike

In [ ]:
df_sub = df[df['sport']=='mountain bike'].copy()
df_sub.reset_index(drop=1,inplace=True)
df_sub

,id,userId,gender,sport,duration,calories,distance,avg_heart_rate,longitude,latitude,altitude,timestamp,heart_rate,speed,url,derived_distance,derived_speed,time_elapsed,validate,avg_alti,change_alti,max_alti,min_alti,diff_alti,avg_speed
0,334121582,13279851,male,mountain bike,4209,585.0,26.410000,123.710,"[5.439254678785801, 5.439083268865943, 5.43904...","[52.01802771538496, 52.017894526943564, 52.017...","[24.8, 25.2, 24.4, 25.6, 25.8, 25.8, 25.0, 24....","[1399222082, 1399222088, 1399222091, 139922209...","[102, 105, 107, 106, 107, 107, 113, 116, 117, ...","[9.018, 12.4884, 12.6324, 12.996, 5.904, 14.73...",https://www.endomondo.com/users/13279851/worko...,"[0.018902475274583067, 0.009400992108153514, 0...","[11.34148516474984, 11.281190529784217, 12.608...","[6, 3, 3, 6, 3, 6, 3, 4, 8, 6, 6, 6, 4, 3, 3, ...",True,9.014000,293.600,39.800,-0.800,40.600,22.590984
1,299896527,13279851,male,mountain bike,5205,969.0,26.710000,149.772,"[5.523203210905194, 5.522962063550949, 5.52271...","[52.025884576141834, 52.02558291144669, 52.025...","[11.4, 10.8, 10.4, 11.0, 11.8, 10.8, 11.0, 11....","[1393003533, 1393003545, 1393003555, 139300356...","[102, 115, 120, 123, 126, 129, 134, 135, 137, ...","[2.34, 13.2552, 16.8876, 17.3628, 15.4692, 19....",https://www.endomondo.com/users/13279851/worko...,"[0.03739824941600433, 0.04238438458152925, 0.0...","[11.219474824801297, 15.25837844935053, 17.691...","[12, 10, 11, 11, 10, 11, 10, 11, 10, 10, 11, 1...",True,31.734800,695.600,70.000,5.800,64.200,17.671215
2,280861813,13279851,male,mountain bike,4016,695.0,19.200000,141.698,"[5.52276567555964, 5.522626284509897, 5.522374...","[52.02590234577656, 52.02570897527039, 52.0253...","[4.6, 5.8, 4.0, 5.4, 4.6, 5.6, 7.2, 6.6, 6.8, ...","[1388422937, 1388422944, 1388422952, 138842296...","[102, 110, 120, 123, 124, 123, 119, 124, 126, ...","[7.218, 16.7112, 16.9308, 16.6356, 17.1576, 10...",https://www.endomondo.com/users/13279851/worko...,"[0.02355992050528061, 0.03855914048723234, 0.0...","[12.116530545572886, 17.351613219254556, 17.69...","[7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...",True,28.206800,491.000,67.200,4.000,63.200,16.613655
3,268331036,13279851,male,mountain bike,4593,977.0,26.400000,138.136,"[5.5180043, 5.5177206, 5.5170584, 5.5165136, 5...","[52.0259099, 52.0255485, 52.0246936, 52.023993...","[7.0, 7.0, 6.8, 6.4, 6.2, 6.0, 5.4, 5.2, 4.8, ...","[1373558941, 1373558949, 1373558966, 137355898...","[129, 138, 140, 134, 132, 129, 127, 131, 130, ...",NaN,https://www.endomondo.com/users/13279851/worko...,"[0.044642134006436636, 0.10533897946360782, 0....","[20.088960302896485, 22.307078004058127, 22.21...","[8, 17, 14, 14, 9, 14, 7, 11, 11, 13, 18, 7, 1...",True,37.839200,424.400,68.000,4.400,63.600,20.892892
4,268331062,13279851,male,mountain bike,6265,1183.0,45.800000,137.708,"[5.5487193, 5.5486971, 5.5482386, 5.5468608, 5...","[52.0036153, 52.0037062, 52.0038875, 52.004203...","[17.0, 16.6, 16.0, 14.8, 13.8, 13.6, 13.2, 13....","[1373466465, 1373466469, 1373466476, 137346649...","[103, 102, 112, 112, 109, 105, 109, 118, 110, ...",NaN,https://www.endomondo.com/users/13279851/worko...,"[0.010232248372871601, 0.03731884619806046, 0....","[9.209023535584441, 19.19254947328824, 21.3223...","[4, 7, 17, 14, 6, 12, 6, 14, 7, 7, 15, 11, 11,...",True,27.525200,515.200,57.400,8.600,48.800,25.993500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3438,326038483,4101,male,mountain bike,4342,979.0,14.595000,144.944,"[10.20356, 10.203871667, 10.204268333, 10.2045...","[55.431498333, 55.431458333, 55.431435, 55.431...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1397864419, 1397864427, 1397864436, 139786444...","[74, 80, 93, 98, 92, 112, 136, 148, 153, 147, ...",NaN,https://www.endomondo.com/users/4101/workouts/...,"[0.020166490148327013, 0.025168122109977153, 0...","[9.074920566747156, 10.06724884399086, 13.9825...","[8, 9, 9, 8, 9, 9, 8, 9, 9, 8, 9, 9, 8, 9, 9, ...",True,34.478000,5

In [ ]:
feature = ['id','distance','change_alti','diff_alti','avg_alti','min_alti','max_alti']
df_cluster = df_sub[feature]

train = df_cluster.drop('id',axis=1)
t= preprocessing.StandardScaler().fit(train)
train=t.transform(train)
train

array([[ 0.004527  , -0.75684474, -0.62913834, -0.55552803, -0.47894004,
        -0.57925992],
       [ 0.02397418,  0.07863727, -0.40621659, -0.4956471 , -0.46043416,
        -0.50557628],
       [-0.46285375, -0.34658566, -0.41566242, -0.50494518, -0.46548122,
        -0.51240788],
       ...,
       [-0.91555186, -0.38607362, -0.60646833, -0.4814069 , -0.43744201,
        -0.5372944 ],
       [-0.13549275,  0.54085418, -0.55923915, -0.46537245, -0.4094028 ,
        -0.50069657],
       [ 0.54985475,  3.68229769,  1.63103323,  0.29598729,  0.11681183,
         0.52293999]])

In [ ]:
%%time
ch_score = []
ss_score = []
meandistortions = []
for k in range(2,15):
    clf = KMeans(n_clusters=k)
    pred = clf.fit_predict(train)
    ch = metrics.calinski_harabasz_score(train,pred)
    ss = metrics.silhouette_score(train,pred)
    ch_score.append(ch)
    ss_score.append(ss)
    meandistortions.append(sum(np.min(cdist(train, clf.cluster_centers_, 'euclidean'), axis=1)) / train.shape[0])

CPU times: user 8.71 s, sys: 12.7 s, total: 21.4 s
Wall time: 6.26 s


In [ ]:
fig = make_subplots(rows=1, cols=3,subplot_titles=("Calinski Harabasz Score", "Elbow Score",'Silhouette Score'))

fig.add_trace(go.Scatter(x=list(range(2,15)), y=ch_score,
                    mode='lines+markers',),row=1,col=1)

fig.add_trace(go.Scatter(x=list(range(2,15)), y=ss_score,
                    mode='lines+markers',
                    name='lines'),row=1,col=3,)

fig.add_trace(go.Scatter(x=list(range(2,15)), y=meandistortions,
                    mode='lines+markers',
                    name='lines'),row=1,col=2,)

fig.update_layout(height=600, width=1200, title_text="Mountain Bike: Number of K",showlegend=False)
fig.show()

In [ ]:
clf = KMeans(n_clusters=3)
pred = clf.fit_predict(train)
pred = [str(i) for i in pred]

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(train)

In [ ]:
projections_df = pd.DataFrame(projections)
projections_df['Cluster']=pred

In [ ]:
fig = px.scatter(
    projections_df, x=0, y=1,
    color='Cluster',
    labels={'color': 'Cluster'},
    category_orders={"Cluster": ["0", "1", "2", "3","4"]}
)

fig.update_layout(height=600, width=1200,title='TSNE: Mountain Biking Route Clustering',font_size=22)
fig.show()

In [ ]:
t = df_cluster.copy()
t['K_means'] = pred
t = t.groupby('K_means').mean()

In [ ]:
x_axis = ['Cluster '+str(i) for i in range(len(t))]
fig = go.Figure(data=[
    go.Bar(name='Distance', x=x_axis, y=t['distance']*100),
    go.Bar(name='Change of Altitude', x=x_axis, y=t['change_alti']),
    go.Bar(name='Diff_Altitude', x=x_axis, y=t['diff_alti']),
    go.Bar(name='Avg_Altitude', x=x_axis, y=t['avg_alti'])
])
# Change the bar mode
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))
fig.update_layout(barmode='group',font_size=22,height=600,width=1200,title = 'Cluster of Mountain Biking')
fig.show()

In [ ]:
df_sub['Cluster']=pred
df_mountain = df_sub.copy()

# Merge

In [ ]:
df_all = df_run.copy()
df_all = df_all.append(df_bike)
df_all = df_all.append(df_mountain)
df_all.reset_index(drop=1,inplace=True)

In [ ]:
df_all

,id,userId,gender,sport,duration,calories,distance,avg_heart_rate,longitude,latitude,altitude,timestamp,heart_rate,speed,url,derived_distance,derived_speed,time_elapsed,validate,avg_alti,change_alti,max_alti,min_alti,diff_alti,avg_speed,Cluster
0,321063199,4969375,male,run,3888,436.0,8.650000,122.104,"[6.8854929, 6.8853678, 6.8851621, 6.8848205, 6...","[52.2226809, 52.222727, 52.2228258, 52.2228606...","[-173.8, -151.2, -161.6, -165.4, -168.6, -172....","[1397079203, 1397079210, 1397079218, 139707922...","[80, 81, 94, 100, 102, 112, 108, 114, 110, 109...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.024692396657788666, 0.02062475820692565, 0....","[12.698946852577029, 10.911666227748753, 10.05...","[7, 8, 8, 8, 7, 8, 8, 8, 8, 7, 8, 8, 8, 8, 7, ...",True,-31.706000,762.000,41.200,-360.800,402.000,8.255823,5
1,303565793,4969375,male,run,4204,577.0,14.370000,146.994,"[6.9144073, 6.9142929, 6.9141539, 6.9140268, 6...","[52.2111711, 52.2112631, 52.2114064, 52.211608...","[57.8, 57.6, 57.0, 56.4, 55.8, 55.2, 54.4, 53....","[1393908533, 1393908541, 1393908549, 139390855...","[60, 62, 92, 92, 132, 150, 150, 159, 159, 161,...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.012866693107454877, 0.018551828170520174, 0...","[5.790011898354694, 7.923126352508781, 8.85172...","[8, 8, 9, 8, 9, 8, 9, 8, 8, 9, 8, 9, 8, 9, 8, ...",True,41.671200,205.800,57.800,23.200,34.600,12.202044,2
2,296982347,4969375,male,run,6503,1025.0,22.220000,134.918,"[6.8678543, 6.8678634, 6.8675429, 6.8672183, 6...","[52.1936673, 52.1934354, 52.1931993, 52.192873...","[35.4, 35.2, 34.6, 34.2, 35.0, 35.2, 34.8, 34....","[1392480163, 1392480176, 1392480189, 139248020...","[75, 101, 116, 120, 124, 126, 127, 129, 126, 1...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.0258024349522409, 0.034169705873918786, 0.0...","[7.1452896790820954, 9.460093775795364, 10.251...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...",True,34.125600,248.000,44.800,24.600,20.200,12.104143,4
3,295890426,4969375,male,run,5118,775.0,16.980000,132.414,"[6.9143328, 6.9146396, 6.9148949, 6.9151568, 6...","[52.2112195, 52.2110264, 52.2108135, 52.210601...","[63.0, 65.2, 66.0, 66.2, 65.8, 65.8, 67.0, 67....","[1392180426, 1392180436, 1392180446, 139218045...","[58, 83, 112, 115, 117, 116, 141, 121, 120, 11...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.03005678592559137, 0.029397223648873875, 0....","[10.820442933212894, 10.685701145251175, 10.36...","[10, 10, 10, 11, 10, 10, 10, 11, 10, 10, 10, 1...",True,45.097200,191.800,67.000,27.000,40.000,11.824599,2
4,294163731,4969375,male,run,3035,445.0,9.670000,125.346,"[6.9133737, 6.9132722, 6.913217, 6.9131066, 6....","[52.2111481, 52.2111209, 52.21119, 52.2112981,...","[61.4, 56.6, 53.2, 49.6, 46.0, 42.4, 38.8, 35....","[1391663545, 1391663551, 1391663557, 139166356...","[56, 62, 75, 80, 80, 96, 107, 108, 113, 117, 1...",NaN,https://www.endomondo.com/users/4969375/workou...,"[0.008947091230656174, 0.009208079362029106, 0...","[5.368254738393704, 6.557825426283524, 7.28499...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6, 6, 6, ...",True,48.845600,252.000,71.200,0.400,70.800,11.395391,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65495,326038483,4101,male,mountain bike,4342,979.0,14.595000,144.944,"[10.20356, 10.203871667, 10.204268333, 10.2045...","[55.431498333, 55.431458333, 55.431435, 55.431...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1397864419, 1397864427, 1397864436, 139786444...","[74, 80, 93, 98, 92, 112, 136, 148, 153, 147, ...",NaN,https://www.endomondo.com/users/4101/workouts/...,"[0.020166490148327013, 0.025168122109977153, 0...","[9.074920566747156, 10.06724884399086, 13.9825...","[8, 9, 9, 8, 9, 9, 8, 9, 9, 8, 9, 9, 8, 9, 9, ...",True,34.478000,509.000,56.000,0.000,56.000,11.493772,2
65496,325108919,4101,male,mountain bike,7991,1668.0,23.995300,132.948,"[10.204013333, 10.204566667, 10.205966667, 10....","[55.43151

In [ ]:
df_all.to_csv('/content/gdrive/My Drive/endomondoHR/Data/endomondo_RouteCluster.csv',index=0)